# Create TRAIN/TEST Datasets for Model Building

## 0. Load Required Packages

In [1]:
### Default Packages
import pandas as pd
import pandas_gbq as gbq
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
%matplotlib inline
import pickle
# import os
import string
# import matplotlib.cm as cm
# from sklearn.externals import joblib


### Pipeline Packages
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer


### Custom Transformers
import Altice_FeatureEngineering as FE                         # Include 'competitive_area' as a feature
from Class_GeneralUtilitiesNonTF import GeneralUtilitiesNonTF  # For BQ-Storage-Python Connection


### Dictionaries for Data Type and Missing Value Imputation
from attribute_dictionary import attribute_dict
from imputation_dictionary import attribute_imputer_dict


### Remove DataConversionWarning
import warnings
warnings.simplefilter('ignore')

## 1. Import TRAIN/TEST Datasets from BigQuery

### Importing TRAIN/TEST Datasets

#### Key Information

In [2]:
# Do Not Change the Below Information Unless Use Others!
bucket     = 'alticeusa-am'
project_id = 'alticeusa-am'
dataset    = 'poc'
auth_file  = 'alticeusa-am-b639e404289b.json'

#Instantiate the util obj that will be used to interact with BQ
util_obj   = GeneralUtilitiesNonTF(project_id  = project_id, 
                                   dataset     = dataset,
                                   bucket_name = bucket,
                                   json_path   = auth_file)

In [3]:
%%time
# TRAIN Dataset
sql_data = ''' SELECT * FROM poc.YST_OPTION1_TRAIN WHERE Segment='FIOS_ONT_G1_4'; ''' 
df_train = util_obj.read_gbq(sql_query = sql_data)
print('\n' + '*'*65 + '\n', f"TRAIN data is SUCCESSFULLY imported! Number of records = {str(len(df_train))}." \
      + '\n' + '*'*65 + '\n')
               
# TEST Dataset
sql_data = ''' SELECT * FROM poc.YST_OPTION1_TEST WHERE Segment='FIOS_ONT_G1_4'; ''' 
df_test  = util_obj.read_gbq(sql_query = sql_data)
print('\n' + '*'*65 + '\n', f"TEST data is SUCCESSFULLY imported! Number of records = {str(len(df_test))}." \
      + '\n' + '*'*65 + '\n')

# For Other Segments
# FIOS_ONT_G1_4
# FIOS_ONT_G4_8
# FIOS_COMP_G1_4
# Other_Segment

Temp table to be created: temp_gcs_06_05_19_03_36_16
**********
Query executed: 
 SELECT * FROM poc.YST_OPTION1_TRAIN WHERE Segment='FIOS_ONT_G1_4'; 
Temp table temp_gcs_06_05_19_03_36_16 created
**********
Info query: SELECT table_name,column_name,data_type,is_nullable FROM poc.INFORMATION_SCHEMA.COLUMNS WHERE table_name='temp_gcs_06_05_19_03_36_16'
Extraction to the tmp_folder_06_05_19_03_36_16 complete
Prefix: tmp_folder_06_05_19_03_36_16/tmp_file_06_05_19_03_36_16
Pulling file: tmp_folder_06_05_19_03_36_16/tmp_file_06_05_19_03_36_16-000000000000.csv.gzip
Pulling file: tmp_folder_06_05_19_03_36_16/tmp_file_06_05_19_03_36_16-000000000001.csv.gzip
Deleted the temp folder tmp_folder_06_05_19_03_36_16

*****************************************************************
 TRAIN data is SUCCESSFULLY imported! Number of records = 106176.
*****************************************************************

Temp table to be created: temp_gcs_06_05_19_03_37_50
**********
Query executed: 
 SELECT *

In [4]:
# Use 'chc_id' as index, and sort by index.
df_train.set_index('chc_id', inplace=True)
df_test.set_index('chc_id', inplace=True)

df_train = df_train.sort_index()
df_test  = df_test.sort_index()

# Drop 'Segment'.
df_train.drop('Segment', axis=1, inplace=True)
df_test.drop('Segment', axis=1, inplace=True)

# TRAIN
train_X  = df_train.drop('status', axis=1).copy()
train_y  = df_train['status']

# TEST
test_X   = df_test.drop('status', axis=1).copy()
test_y   = df_test['status']

# Sample Size
print('*'*50 + '\nTRAIN vs TEST Datasets\n' + '*'*50)
print('Competitive Area: ', df_train.competitive_area.unique())
print('The Shape of TRAIN Data: ' + str(df_train.shape))
print('The Shape of TEST Data:  ' + str(df_test.shape))

## Churn Rate by Sample Type
print('\n' + '*'*50 + '\nOverall Churn Rate\n' + '*'*50)
print('TRAIN: ', df_train.status.value_counts(normalize=True)[1].round(3))
print('TEST:  ', df_test.status.value_counts(normalize=True)[1].round(3), '\n')

# print(train_X.index)
# print(train_y.index)
# print(test_X.index)
# print(test_y.index)

**************************************************
TRAIN vs TEST Datasets
**************************************************
Competitive Area:  ['Fios ONT Competitive Area']
The Shape of TRAIN Data: (106176, 1037)
The Shape of TEST Data:  (102062, 1037)

**************************************************
Overall Churn Rate
**************************************************
TRAIN:  0.025
TEST:   0.033 



## 2. Pre-Process TRAIN/TEST Datasets

### Pre-Processing Data

In [5]:
%%time

# (1) Make a Pipeline and Instantiate
Pipe_PP = Pipeline([
                    ('DataType', FE.Use_DefaultDataType(default_dtypes=attribute_dict)),
                    ('Missing', FE.Remove_MissingFeatures(missing_threshold=0.99)), 
                    ('Constant1', FE.Remove_ConstantFeatures(unique_threshold=1, missing_threshold=0.00)), 
                    ('Correlated1', FE.Remove_CorrelatedFeatures(correlation_threshold=0.99)), 
                    ('Duplicate', FE.Remove_DuplicateFeatures()),
                    ('Imputer', FE.Use_DefaultImputer(default_imputers=attribute_imputer_dict, default_dtypes=attribute_dict)),
                    ('Constant2', FE.Remove_ConstantFeatures(unique_threshold=1, missing_threshold=0.00)), 
                    ('Correlated2', FE.Remove_CorrelatedFeatures(correlation_threshold=0.90))
                  ])

# 'Constant2' is added to handle (1) unique value = 0 and (2) default imputation value = 0.
# 'Correlated2' is added to further remove correlated features after impuation.


# (2) fit()
Pipe_PP.fit(train_X, train_y)


# (3) transform()
train_X_PP = Pipe_PP.transform(train_X)
test_X_PP  = Pipe_PP.transform(test_X)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Transformation\n' + '*'*50)
print('TRAIN: Before Transformation:' + str(train_X.shape))
print('TRAIN: After Transformation: ' + str(train_X_PP.shape))
print('TEST:  After Transformation: ' + str(test_X_PP.shape))

**************************************************
Pre-Processing: Use_DefaultDataType
**************************************************
- It will convert data types into default ones.

**************************************************
Pre-Processing: Remove_MissingFeatures
**************************************************
- It will remove features with a high missing pct.

**************************************************
Pre-Processing: Remove_ConstantFeatures
**************************************************
- It will remove features with 1 unique value(s).

**************************************************
Pre-Processing: Remove_CorrelatedFeatures
**************************************************
- It will work on Numerical Features Only, doing nothing on Categorical Features.
- It may take 10+ minutes. Be patient!

**************************************************
Pre-Processing: Remove_DuplicateFeatures
**************************************************
- It may take 10+ 

### Creating New Features

In [6]:
%%time

# (1) Make a Pipeline and Instantiate
Pipe_NF = Pipeline([
                    ('Imputer', FE.Use_DefaultImputer(default_imputers=attribute_imputer_dict, default_dtypes=attribute_dict)),
                    ('NewFeatures', FE.FeatureMaker())
                  ])

# 'Imputer' is added to handle missing values
# 'NewFeature' is added to create new features


# (2) fit()
Pipe_NF.fit(train_X, train_y)


# (3) transform()
train_X_NF = Pipe_NF.transform(train_X)
test_X_NF  = Pipe_NF.transform(test_X)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Transformation\n' + '*'*50)
print('TRAIN: Before Transformation:' + str(train_X.shape))
print('TRAIN: After Transformation: ' + str(train_X_NF.shape))
print('TEST:  After Transformation: ' + str(test_X_NF.shape))
print('\n' + '*'*50 + '\nNewly Created Features\n' + '*'*50 + '\n', 
      Pipe_NF.named_steps['NewFeatures'].features_new_)

**************************************************
Pre-Processing: Use_DefaultImputere
**************************************************
- It will append default imputation values to missings.


**************************************************
Before vs After Transformation
**************************************************
TRAIN: Before Transformation:(106176, 1036)
TRAIN: After Transformation: (106176, 16)
TEST:  After Transformation: (102062, 16)

**************************************************
Newly Created Features
**************************************************
 ['grp_tenure_3m', 'grp_tenure_1m', 'grp_tenure_6m', 'grp_payment_method', 'grp_payment_25dollar', 'grp_payment_10dollar', 'grp_payment_change_5dollar', 'grp_payment_change_10dollar', 'grp_payment_change_2pct', 'grp_payment_change_5pct', 'ratio_payment_income', 'grp_payment_income', 'grp_call_csc', 'grp_call_bill', 'grp_call_csr', 'grp_call_tsr']
CPU times: user 9min 7s, sys: 5min 46s, total: 14min 54s
Wall time: 

### Combining Processed and New Features

In [7]:
%%time

# Create Datasets that Consist of Pre-processed and New Features.
df_train_NF_PP = pd.concat([train_y.to_frame(), train_X_NF, train_X_PP], axis=1)
df_test_NF_PP  = pd.concat([test_y.to_frame(), test_X_NF, test_X_PP], axis=1)

# Save Data for Feature Engineering
# Pre-processed data with new features
# df_train_NF_PP.to_pickle('data_new/FIOS_ONT_G1_4_train_PP.pkl')
# df_test_NF_PP.to_pickle('data_new/FIOS_ONT_G1_4_test_PP.pkl')

print(df_train_NF_PP.shape)
print(df_test_NF_PP.shape)

(106176, 790)
(102062, 790)
CPU times: user 900 ms, sys: 672 ms, total: 1.57 s
Wall time: 1.57 s


## 3. Feature Engineering

In [8]:
# Use Pre-Processed Data as new TRAIN/TEST Datasets
train_X = pd.concat([train_X_NF, train_X_PP], axis=1)
test_X  = pd.concat([test_X_NF, test_X_PP], axis=1)
print(train_X.shape)
print(test_X.shape)

(106176, 789)
(102062, 789)


#### Re-Use Pre-Processed Data! Pre-Processing Takes an 1 Hour!

In [9]:
# df_train = pd.read_pickle('data_new/FIOS_ONT_G1_4_train_PP.pkl')
# df_test  = pd.read_pickle('data_new/FIOS_ONT_G1_4_test_PP.pkl')

# # TRAIN
# train_X  = df_train.drop('status', axis=1).copy()
# train_y  = df_train['status']

# # TEST
# test_X   = df_test.drop('status', axis=1).copy()
# test_y   = df_test['status']

# # Sample Size
# print('*'*50 + '\nTRAIN vs TEST Datasets\n' + '*'*50)
# print('The Shape of TRAIN Data: ' + str(df_train.shape))
# print('The Shape of TEST Data:  ' + str(df_test.shape))

# ## Churn Rate by Sample Type
# print('\n' + '*'*50 + '\nOverall Churn Rate\n' + '*'*50)
# print('TRAIN: ', df_train.status.value_counts(normalize=True)[1].round(4))
# print('TEST:  ', df_test.status.value_counts(normalize=True)[1].round(4), '\n')

### FE for NUMERICAL Features

#### Create Pipelines by Transformation

In [10]:
corr_target   = 0.005 # To remove features with low correlation with target: abs(Corr w. Target) < 0.005
corr_features = 0.75  # To remove features with high correlation with other features: abs(Corr) > 0.75

Pipe_Original    = Pipeline([('Original', FE.PassTransformer(prefix='Original')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Tree        = Pipeline([('Tree', FE.DecisionTreeDiscretizer(cv=3,  corr_threshold_target=0.001, prefix='Tree_NUM')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Standard    = Pipeline([('Standard', FE.StandardScaler_DF(prefix='Standard')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Robust      = Pipeline([('Robust', FE.RobustScaler_DF(prefix='Robust', quantile_range=(5.0, 95.0))),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_MinMax      = Pipeline([('MinMax', FE.MinMaxScaler_DF(prefix='MinMax', feature_range=(0, 1))),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_MaxAbs     = Pipeline([('MaxAbs', FE.MaxAbsScaler_DF(prefix='MaxAbs')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Norm       = Pipeline([('Norm', FE.Normalizer_DF(prefix='Norm', norm='l1')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Quantile   = Pipeline([('Quantile', FE.QuantileTransformer_DF(prefix='Quantile', n_quantiles=100, random_state=0)),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_KBin       = Pipeline([('KBin', FE.KBinsDiscretizer_DF(prefix='KBin', n_bins=10, encode='ordinal')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Binary     = Pipeline([('Binary', FE.Binarizer_DF(prefix='Binary', threshold=0)),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Log1p      = Pipeline([('Log1p', FE.Log1pTransformer(prefix='Log1p')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Sqrt       = Pipeline([('Sqrt', FE.SqrtTransformer(prefix='Sqrt')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Reciprocal = Pipeline([('Reciprocal', FE.ReciprocalTransformer(prefix='Reciprocal')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

'Remove_IrrelevantCollinearFeatures' requires target y.
'DecisionTreeDiscretizer' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.


#### Create/Use a Meta-Transformer
#### abs(correlation ) among Features <= 0.75

In [11]:
%%time
corr_target   = 0.01  # To remove features with low correlation with target: abs(Corr w. Target) < 0.01
corr_features = 0.75  # To remove features with high correlation with other features: abs(Corr) > 0.75

# (1) Make a Pipeline in Parallel/Sequence and Instantiate 
# Custom Transformers in Parallel for NUMERICAL Features
Pipe_FU          =  FE.FeatureUnion_DF([
                                        ('Original_Pipe', Pipe_Original),     
                                        ('Tree_Pipe', Pipe_Tree),         
                                        ('Standard_Pipe', Pipe_Standard), 
                                        ('Robust_Pipe', Pipe_Robust), 
                                        ('Quantile_Pipe', Pipe_Quantile), 
                                        ('Binary_Pipe', Pipe_Binary),         
                                        ('MinMax_Pipe', Pipe_MinMax), 
                                        ('MaxAbs_Pipe', Pipe_MaxAbs),                                                                                                                                
                                        ('Norm_Pipe', Pipe_Norm), 
                                        ('KBin', Pipe_KBin),     
                                        ('Log1p_Pipe', Pipe_Log1p), 
                                        ('Sqrt_Pipe', Pipe_Sqrt), 
                                        ('Reciprocal_Pipe', Pipe_Reciprocal)
                                      ])
# Custom Transformers in Sequence for NUMERICAL Features
NUM_Pipe          = Pipeline([
                              ('Selector', FE.FeatureSelector_NUM()),
                              ('FU_Pipe', Pipe_FU),
                              ('IrrelevantCollinear', FE.Remove_IrrelevantCollinearFeatures(
                               corr_threshold_target=corr_target, corr_threshold_features=corr_features))
                             ])


# (2) fit()
NUM_Pipe.fit(train_X, train_y)


# (3) transform()
train_X_FE = NUM_Pipe.transform(train_X)
test_X_FE  = NUM_Pipe.transform(test_X)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Feature Engineering (FE)\n' + '*'*50)
print('TRAIN: Before FE:' + str(train_X.shape))
print('TRAIN: After FE: ' + str(train_X_FE.shape))
print('TEST:  After FE: ' + str(test_X_FE.shape))

'Remove_IrrelevantCollinearFeatures' requires target y.

****************************************************************************************************
Total NUMERICAL Features: 402 features examined
Irrelevant Features     : 255 features dropped due to abs(correlation ) <= 0.005 with target
Collinear Features      : 32 features dropped due to abs(correlation ) > 0.75 with other features
****************************************************************************************************

****************************************************************************************************
Total NUMERICAL Features: 345 features examined
Irrelevant Features     : 115 features dropped due to abs(correlation ) <= 0.005 with target
Collinear Features      : 16 features dropped due to abs(correlation ) > 0.75 with other features
****************************************************************************************************

***********************************************************

In [12]:
p_list          = [.01, .05, .1, .2, .3, .4, .6, .7, .8, .9, .95, .99]
flag_NUM        = train_X.select_dtypes(exclude=[object, 'category']).columns.tolist()
corr_train      = train_X[flag_NUM].apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr_train_all  = train_X_FE.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)

corr_test       = test_X[flag_NUM].apply(lambda x: x.corr(test_y)).to_frame().describe(percentiles=p_list)
corr_test_all   = test_X_FE.apply(lambda x: x.corr(test_y)).to_frame().describe(percentiles=p_list)

corr_all         = pd.concat([corr_train, corr_train_all, corr_test, corr_test_all], axis=1)
corr_all.columns = ['TRAIN_Original', 'TRAIN_All', 'TEST_Original', 'TEST_All']
print('\n' + '*'*50 + '\nCorrelation Summary: TRAIN vs TEST\n' + '*'*50)
print('abs(correlation) among Features <= 0.75')
corr_all


**************************************************
Correlation Summary: TRAIN vs TEST
**************************************************
abs(correlation) among Features <= 0.75


,TRAIN_Original,TRAIN_All,TEST_Original,TEST_All
count,402.000000,206.000000,402.000000,206.000000
mean,0.000722,0.016547,0.001845,0.010000
std,0.008148,0.015291,0.009021,0.013968
min,-0.036538,-0.022044,-0.045534,-0.027004
1%,-0.023006,-0.018947,-0.017500,-0.019239
5%,-0.010472,-0.014204,-0.007537,-0.007335
10%,-0.006377,-0.010886,-0.005932,-0.003594
20%,-0.004024,0.010757,-0.003819,-0.000497
30%,-0.002742,0.012620,-0.002410,0.001682
40%,-0.001234,0.014707,-0.001070,0.004006


In [13]:
df_train_FE_NUM = train_y.to_frame().\
                 merge(train_X_FE, how='inner', left_index=True, right_index=True)

df_test_FE_NUM  = test_y.to_frame().\
                  merge(test_X_FE, how='inner', left_index=True, right_index=True)

#### abs(correlation ) among Features <= 0.50

In [14]:
# corr_target   = 0.01  # To remove features with low correlation with target: abs(Corr w. Target) < 0.01
# corr_features = 0.50  # To remove features with high correlation with other features: abs(Corr) > 0.50

# (1) Instantiate
RICF = FE.Remove_IrrelevantCollinearFeatures(corr_threshold_target=0.01, corr_threshold_features=0.50)

# (2) Fit
RICF.fit(train_X_FE, train_y)

# (3) Transform
train_X_FE_50Corr = RICF.transform(train_X_FE)
test_X_FE_50Corr  = RICF.transform(test_X_FE)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Feature Engineering (FE)' + '\n' + '*'*50)
print('TRAIN: Before FE:' + str(train_X_FE.shape))
print('TRAIN: After FE: ' + str(train_X_FE_50Corr.shape))
print('TEST:  After FE: ' + str(test_X_FE_50Corr.shape))

'Remove_IrrelevantCollinearFeatures' requires target y.

****************************************************************************************************
Total NUMERICAL Features: 206 features examined
Irrelevant Features     : 0 features dropped due to abs(correlation ) <= 0.01 with target
Collinear Features      : 89 features dropped due to abs(correlation ) > 0.5 with other features
****************************************************************************************************

**************************************************
Before vs After Feature Engineering (FE)
**************************************************
TRAIN: Before FE:(106176, 206)
TRAIN: After FE: (106176, 117)
TEST:  After FE: (102062, 117)


In [15]:
p_list          = [.01, .05, .1, .2, .3, .4, .6, .7, .8, .9, .95, .99]
flag_NUM        = train_X.select_dtypes(exclude=[object, 'category']).columns.tolist()
corr_train      = train_X[flag_NUM].apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr_train_all  = train_X_FE_50Corr.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)

corr_test       = test_X[flag_NUM].apply(lambda x: x.corr(test_y)).to_frame().describe(percentiles=p_list)
corr_test_all   = test_X_FE_50Corr.apply(lambda x: x.corr(test_y)).to_frame().describe(percentiles=p_list)

corr_all         = pd.concat([corr_train, corr_train_all, corr_test, corr_test_all], axis=1)
corr_all.columns = ['TRAIN_Original', 'TRAIN_All', 'TEST_Original', 'TEST_All']
print('\n' + '*'*50 + '\nCorrelation Summary: TRAIN vs TEST\n' + '*'*50)
print('abs(correlation) among Features <= 0.50')
corr_all


**************************************************
Correlation Summary: TRAIN vs TEST
**************************************************
abs(correlation ) among Features <= 0.50


,TRAIN_Original,TRAIN_All,TEST_Original,TEST_All
count,402.000000,117.000000,402.000000,117.000000
mean,0.000722,0.016283,0.001845,0.006824
std,0.008148,0.014900,0.009021,0.012284
min,-0.036538,-0.022044,-0.045534,-0.023111
1%,-0.023006,-0.017671,-0.017500,-0.019110
5%,-0.010472,-0.012448,-0.007537,-0.010099
10%,-0.006377,-0.010599,-0.005932,-0.003322
20%,-0.004024,0.010747,-0.003819,-0.000804
30%,-0.002742,0.012595,-0.002410,0.000461
40%,-0.001234,0.014208,-0.001070,0.002791


In [16]:
df_train_FE_NUM_50Corr = train_y.to_frame().\
                         merge(train_X_FE_50Corr, how='inner', left_index=True, right_index=True)

df_test_FE_NUM_50Corr  = test_y.to_frame().\
                         merge(test_X_FE_50Corr, how='inner', left_index=True, right_index=True)

### FE for CATEGORICAL Features

#### Create Pipelines by Transformation

In [17]:
corr_target   = 0.005 # To remove features with low correlation with target: abs(Corr w. Target) < 0.005
corr_features = 0.75  # To remove features with high correlation with other features: abs(Corr) > 0.75

fe_1st           = ['grp_tenure_3m', 'grp_payment_method', \
                    'grp_payment_25dollar', 'grp_payment_change_10dollar', 'grp_payment_change_5pct', \
                    'grp_payment_income', 'grp_call_csc', 'grp_call_bill', \
                    'grp_call_csr', 'grp_call_tsr']
fe_2nd           = fe_1st + ['income_demos', 'ethnic', 'age_demos', 'archetype']
fe_group         = ['census', 'cleansed_city', 'trunk', 'hub']

Pipe_OHE         = Pipeline([('OHE', FE.UniversalCategoryEncoder(encoding_method='ohe')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_PCT         = Pipeline([('PCT', FE.UniversalCategoryEncoder(encoding_method='pct', prefix='PCT')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_COUNT       = Pipeline([('COUNT', FE.UniversalCategoryEncoder(encoding_method='count', prefix='COUNT')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_ORDINAL     = Pipeline([('ORDINAL', FE.UniversalCategoryEncoder(encoding_method='ordinal', prefix='ORDINAL')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Y_MEAN      = Pipeline([('Y_MEAN', FE.UniversalCategoryEncoder(encoding_method='y_mean', prefix='Y_MEAN')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Y_LOG_RATIO = Pipeline([('Y_LOG_RATIO', FE.UniversalCategoryEncoder(encoding_method='y_log_ratio', prefix='Y_LOG_RATIO')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Y_RATIO     = Pipeline([('Y_RATIO', FE.UniversalCategoryEncoder(encoding_method='y_ratio', prefix='Y_RATIO')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Aggregation = Pipeline([('Aggregation', FE.FeatureAggregator(features_grouping=fe_group, correlation_threshold=0.01)),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

Pipe_Tree        = Pipeline([('ORDINAL', FE.UniversalCategoryEncoder(encoding_method='ordinal')),
                             ('Tree', FE.DecisionTreeDiscretizer(cv=3,  corr_threshold_target=0.001, prefix='Tree_CAT')),
                             ('Correlated', FE.Remove_IrrelevantCollinearFeatures(
                              corr_threshold_target=corr_target, corr_threshold_features=corr_features))])

'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'ordinal' encoding requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'y_mean' encoding requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'y_log_ratio' encoding requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'y_ratio' encoding requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'FeatureAggregator' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.
'ordinal' encoding requires target y.
'DecisionTreeDiscretizer' requires target y.
'Remove_IrrelevantCollinearFeatures' requires target y.


#### Create/Use a Meta-Transformer
#### abs(correlation ) among Features <= 0.75

In [18]:
%%time
corr_target   = 0.01  # To remove features with low correlation with target: abs(Corr w. Target) < 0.01
corr_features = 0.75  # To remove features with high correlation with other features: abs(Corr) > 0.75

# (1) Make a Pipeline in Parallel/Sequence and Instantiate 
# Custom Transformers in Parallel for CATEGORICAL Features
Pipe_FU          =  FE.FeatureUnion_DF([
                    ('OHE_Pipe', Pipe_OHE),
                    ('PCT_Pipe', Pipe_PCT),
                    ('COUNT_Pipe', Pipe_COUNT),
                    ('ORDINAL_Pipe', Pipe_ORDINAL),
                    ('Y_MEAN_Pipe', Pipe_Y_MEAN),
                    ('Y_LOG_RATIO_Pipe', Pipe_Y_LOG_RATIO),
                    ('Y_RATIO_Pipe', Pipe_Y_RATIO),
                    ('Aggregation_Pipe', Pipe_Aggregation),
                    ('Tree_Pipe', Pipe_Tree)
                    ])

# Custom Transformers in Sequence for CATEGORICAL Features
CAT_Pipe          = Pipeline([
                    ('Interaction', FE.FeatureInteractionTransformer(features_1st=fe_1st, features_2nd=fe_2nd)),
                    ('RareCategory', FE.RareCategoryEncoder(category_min_pct=0.01, category_max_count=30)),
                    ('FU_Pipe', Pipe_FU),
                    ('IrrelevantCollinear', FE.Remove_IrrelevantCollinearFeatures(
                     corr_threshold_target=corr_target, corr_threshold_features=corr_features))
                    ])


# (2) fit()
CAT_Pipe.fit(train_X, train_y)


# (3) transform()
train_X_FE = CAT_Pipe.transform(train_X)
test_X_FE  = CAT_Pipe.transform(test_X)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Feature Engineering (FE)\n' + '*'*50)
print('TRAIN: Before FE:' + str(train_X.shape))
print('TRAIN: After FE: ' + str(train_X_FE.shape))
print('TEST:  After FE: ' + str(test_X_FE.shape))

'Remove_IrrelevantCollinearFeatures' requires target y.

****************************************************************************************************
Total NUMERICAL Features: 3815 features examined
Irrelevant Features     : 2260 features dropped due to abs(correlation ) <= 0.005 with target
Collinear Features      : 887 features dropped due to abs(correlation ) > 0.75 with other features
****************************************************************************************************

****************************************************************************************************
Total NUMERICAL Features: 464 features examined
Irrelevant Features     : 227 features dropped due to abs(correlation ) <= 0.005 with target
Collinear Features      : 147 features dropped due to abs(correlation ) > 0.75 with other features
****************************************************************************************************

*******************************************************

In [19]:
p_list          = [.01, .05, .1, .2, .3, .4, .6, .7, .8, .9, .95, .99]
corr_train_all  = train_X_FE.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr_test_all   = test_X_FE.apply(lambda x: x.corr(test_y)).to_frame().describe(percentiles=p_list)

corr_all         = pd.concat([corr_train_all, corr_test_all], axis=1)
corr_all.columns = ['TRAIN_All', 'TEST_All']
print('\n' + '*'*50 + '\nCorrelation Summary: TRAIN vs TEST\n' + '*'*50)
print('abs(correlation) among Features <= 0.75')
corr_all


**************************************************
Correlation Summary: TRAIN vs TEST
**************************************************
abs(correlation ) among Features <= 0.75


,TRAIN_All,TEST_All
count,391.000000,386.000000
mean,0.010214,0.007751
std,0.014178,0.014186
min,-0.024233,-0.040173
1%,-0.020414,-0.022277
5%,-0.014528,-0.014605
10%,-0.012842,-0.010307
20%,-0.010225,-0.002554
30%,0.010540,0.000745
40%,0.011267,0.004052


#### abs(correlation ) among Features <= 0.50

In [20]:
# corr_target   = 0.01  # To remove features with low correlation with target: abs(Corr w. Target) < 0.01
# corr_features = 0.50  # To remove features with high correlation with other features: abs(Corr) > 0.50

# (1) Instantiate
RICF = FE.Remove_IrrelevantCollinearFeatures(corr_threshold_target=0.01, corr_threshold_features=0.50)

# (2) Fit
RICF.fit(train_X_FE, train_y)

# (3) Transform
train_X_FE_50Corr = RICF.transform(train_X_FE)
test_X_FE_50Corr  = RICF.transform(test_X_FE)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Feature Engineering (FE)' + '*'*50)
print('TRAIN: Before FE:' + str(train_X_FE.shape))
print('TRAIN: After FE: ' + str(train_X_FE_50Corr.shape))
print('TEST:  After FE: ' + str(test_X_FE_50Corr.shape))

'Remove_IrrelevantCollinearFeatures' requires target y.

****************************************************************************************************
Total NUMERICAL Features: 391 features examined
Irrelevant Features     : 0 features dropped due to abs(correlation ) <= 0.01 with target
Collinear Features      : 215 features dropped due to abs(correlation ) > 0.5 with other features
****************************************************************************************************

**************************************************
Before vs After Feature Engineering (FE)**************************************************
TRAIN: Before FE:(106176, 391)
TRAIN: After FE: (106176, 176)
TEST:  After FE: (102062, 176)


In [21]:
p_list          = [.01, .05, .1, .2, .3, .4, .6, .7, .8, .9, .95, .99]
corr_train_all  = train_X_FE_50Corr.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr_test_all   = test_X_FE_50Corr.apply(lambda x: x.corr(test_y)).to_frame().describe(percentiles=p_list)

corr_all         = pd.concat([corr_train_all, corr_test_all], axis=1)
corr_all.columns = ['TRAIN_All', 'TEST_All']
print('\n' + '*'*50 + '\nCorrelation Summary: TRAIN vs TEST\n' + '*'*50)
print('abs(correlation) among Features <= 0.50')
corr_all


**************************************************
Correlation Summary: TRAIN vs TEST
**************************************************
abs(correlation ) among Features <= 0.50


,TRAIN_All,TEST_All
count,176.000000,173.000000
mean,0.009022,0.005629
std,0.014403,0.012550
min,-0.021032,-0.040173
1%,-0.017761,-0.022310
5%,-0.014766,-0.013395
10%,-0.013404,-0.010332
20%,-0.010516,-0.003053
30%,0.010149,-0.000643
40%,0.010891,0.001895


### Save Feature-Engineered Data to Storage

#### Re-Use Transformed/Selected Featuers Again!

In [22]:
# abs(correlation ) among Features <= 0.75
df_train = pd.concat([df_train_FE_NUM, train_X_FE], axis=1)
df_test  = pd.concat([df_test_FE_NUM, test_X_FE], axis=1)

# TRAIN
train_X  = df_train.drop('status', axis=1).copy()
train_y  = df_train['status']

# TEST
test_X   = df_test.drop('status', axis=1).copy()
test_y   = df_test['status']

# Sample Size
print('*'*50 + '\nTRAIN vs TEST Datasets\n' + '*'*50)
print('The Shape of TRAIN Data: ' + str(df_train.shape))
print('The Shape of TEST Data:  ' + str(df_test.shape))

## Churn Rate by Sample Type
print('\n' + '*'*50 + '\nOverall Churn Rate\n' + '*'*50)
print('TRAIN: ', df_train.status.value_counts(normalize=True)[1].round(4))
print('TEST:  ', df_test.status.value_counts(normalize=True)[1].round(4), '\n')

**************************************************
TRAIN vs TEST Datasets
**************************************************
The Shape of TRAIN Data: (106176, 598)
The Shape of TEST Data:  (102062, 598)

**************************************************
Overall Churn Rate
**************************************************
TRAIN:  0.0252
TEST:   0.0332 



#### abs(correlation ) among Features <= 0.75

In [23]:
# corr_target   = 0.01  # To remove features with low correlation with target: abs(Corr w. Target) < 0.01
# corr_features = 0.75  # To remove features with high correlation with other features: abs(Corr) > 0.75

# (1) Instantiate
RICF = FE.Remove_IrrelevantCollinearFeatures(corr_threshold_target=0.01, corr_threshold_features=0.75)

# (2) Fit
RICF.fit(train_X, train_y)

# (3) Transform
train_X_FE = RICF.transform(train_X)
test_X_FE  = RICF.transform(test_X)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Feature Engineering (FE)' + '\n' + '*'*50)
print('TRAIN: Before FE:' + str(train_X.shape))
print('TRAIN: After FE: ' + str(train_X_FE.shape))
print('TEST:  After FE: ' + str(test_X_FE.shape))

# abs(correlation) among Features <= 0.50
df_train_FE = pd.concat([train_y, train_X_FE], axis=1)
df_test_FE  = pd.concat([test_y, test_X_FE], axis=1)

df_train_FE.to_pickle('data_processed/FIOS_ONT_G1_4_train_FE_75Corr.pkl')
df_test_FE.to_pickle('data_processed/FIOS_ONT_G1_4_train_FE_75Corr.pkl')

'Remove_IrrelevantCollinearFeatures' requires target y.

****************************************************************************************************
Total NUMERICAL Features: 597 features examined
Irrelevant Features     : 0 features dropped due to abs(correlation ) <= 0.01 with target
Collinear Features      : 12 features dropped due to abs(correlation ) > 0.75 with other features
****************************************************************************************************

**************************************************
Before vs After Feature Engineering (FE)
**************************************************
TRAIN: Before FE:(106176, 597)
TRAIN: After FE: (106176, 585)
TEST:  After FE: (102062, 585)


#### abs(correlation ) among Features <= 0.50

In [24]:
# corr_target   = 0.01  # To remove features with low correlation with target: abs(Corr w. Target) < 0.01
# corr_features = 0.50  # To remove features with high correlation with other features: abs(Corr) > 0.50

# (1) Instantiate
RICF = FE.Remove_IrrelevantCollinearFeatures(corr_threshold_target=0.01, corr_threshold_features=0.50)

# (2) Fit
RICF.fit(train_X, train_y)

# (3) Transform
train_X_FE = RICF.transform(train_X)
test_X_FE  = RICF.transform(test_X)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Feature Engineering (FE)' + '\n' + '*'*50)
print('TRAIN: Before FE:' + str(train_X.shape))
print('TRAIN: After FE: ' + str(train_X_FE.shape))
print('TEST:  After FE: ' + str(test_X_FE.shape))

# abs(correlation) among Features <= 0.50
df_train_FE = pd.concat([train_y, train_X_FE], axis=1)
df_test_FE  = pd.concat([test_y, test_X_FE], axis=1)

df_train_FE.to_pickle('data_processed/FIOS_ONT_G1_4_train_FE_50Corr.pkl')
df_test_FE.to_pickle('data_processed/FIOS_ONT_G1_4_train_FE_50Corr.pkl')

'Remove_IrrelevantCollinearFeatures' requires target y.

****************************************************************************************************
Total NUMERICAL Features: 597 features examined
Irrelevant Features     : 0 features dropped due to abs(correlation ) <= 0.01 with target
Collinear Features      : 326 features dropped due to abs(correlation ) > 0.5 with other features
****************************************************************************************************

**************************************************
Before vs After Feature Engineering (FE)
**************************************************
TRAIN: Before FE:(106176, 597)
TRAIN: After FE: (106176, 271)
TEST:  After FE: (102062, 271)


#### abs(correlation ) among Features <= 0.30

In [25]:
# corr_target   = 0.01  # To remove features with low correlation with target: abs(Corr w. Target) < 0.01
# corr_features = 0.50  # To remove features with high correlation with other features: abs(Corr) > 0.30

# (1) Instantiate
RICF = FE.Remove_IrrelevantCollinearFeatures(corr_threshold_target=0.01, corr_threshold_features=0.30)

# (2) Fit
RICF.fit(train_X, train_y)

# (3) Transform
train_X_FE = RICF.transform(train_X)
test_X_FE  = RICF.transform(test_X)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Feature Engineering (FE)' + '\n' + '*'*50)
print('TRAIN: Before FE:' + str(train_X.shape))
print('TRAIN: After FE: ' + str(train_X_FE.shape))
print('TEST:  After FE: ' + str(test_X_FE.shape))

# abs(correlation) among Features <= 0.50
df_train_FE = pd.concat([train_y, train_X_FE], axis=1)
df_test_FE  = pd.concat([test_y, test_X_FE], axis=1)

df_train_FE.to_pickle('data_processed/FIOS_ONT_G1_4_train_FE_30Corr.pkl')
df_test_FE.to_pickle('data_processed/FIOS_ONT_G1_4_train_FE_30Corr.pkl')

'Remove_IrrelevantCollinearFeatures' requires target y.

****************************************************************************************************
Total NUMERICAL Features: 597 features examined
Irrelevant Features     : 0 features dropped due to abs(correlation ) <= 0.01 with target
Collinear Features      : 490 features dropped due to abs(correlation ) > 0.3 with other features
****************************************************************************************************

**************************************************
Before vs After Feature Engineering (FE)
**************************************************
TRAIN: Before FE:(106176, 597)
TRAIN: After FE: (106176, 107)
TEST:  After FE: (102062, 107)


#### gsutil cp SEG*.pkl gs://alticeusa-am_tkim/pre-processed_data/new